# Linear regression

## Preparación

### Importación de librerías

In [1]:
import numpy as np # linear algebra
import pandas as pd  # data management (dataframes)
import matplotlib.pyplot as plt
import seaborn as sns  # plotting

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree, export_text
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

#from sklearn import metrics
#from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.inspection import permutation_importance

# stacking
from sklearn.ensemble import VotingRegressor, StackingRegressor

from sklearn.linear_model  import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer

#from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

# others
from mltools import classification_tools as CT
from mltools import model_tools as MT

In [ ]:
### Load libraries ###

# interactive plotting
#%matplotlib inline
#%config InlineBackend.figure_format = 'svg' # ‘png’, ‘retina’, ‘jpeg’, ‘svg’, ‘pdf’

# plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()


# Data management libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Machine learning libraries
import math
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer

from sklearn import set_config
set_config(display='diagram')

# others
from statsmodels.stats.outliers_influence import variance_inflation_factor
from mltools import regression_tools as RT
from mltools.regression_tools import LinearRegressor
from mltools import model_tools as MT

### Carga de datos

In [2]:
df = pd.read_csv('../df.csv')

### Dividir en train y test

In [3]:
OUTPUT = 'UTIL_VALUE'
X = df.drop(columns=[OUTPUT])
y = df[OUTPUT]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,  #percentage of test data
                                                    random_state=0) #seed for replication

## Modelos

### Linear Regression

In [4]:
INPUTS_LR_NUM = ["age","bmi"]
INPUTS_LR_CAT = ["sex","children","smoker","region"]
INPUTS_LR = INPUTS_LR_NUM + INPUTS_LR_CAT

# Prepare the numeric variables by imputing by scaling
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# Prepare the categorical variables by encoding the categories
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore',drop='first'))])

# Create a preprocessor to perform the steps defined above
preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, INPUTS_LR_NUM),
        ('cat', categorical_transformer, INPUTS_LR_CAT)
        ])

pipe = Pipeline(steps=[('Prep',preprocessor), # Preprocess the variables when training the model 
                        ('LinReg',LinearRegressor())]) 


# We use Grid Search Cross Validation to find the best parameter for the model in the grid defined 
nFolds = 10
param = {}
LR_fit = GridSearchCV(estimator=pipe, # Structure of the model to use
                        param_grid=param, # Defined grid to search in
                        n_jobs=-1, # Number of cores to use (parallelize)
                        cv=nFolds) # Number of Folds 
LR_fit.fit(X_train[INPUTS_LR], y_train) # Search in grid

Ridge(alpha=0.1)

In [ ]:
LR_fit.best_estimator_['LinReg'].summary(LR_fit.best_estimator_['Prep'].get_feature_names_out())

In [ ]:
X_processed = LR_fit.best_estimator_['Prep'].transform(X_train)
coefnames = LR_fit.best_estimator_['Prep'].get_feature_names_out(INPUTS_LR)
#Identify correlated variables
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_processed, i) for i in range(X_processed.shape[1])]
vif["features"] = coefnames
vif.round(1)

In [ ]:
dfTR_eval['LR_pred'] = LR_fit.predict(X_train)
dfTS_eval['LR_pred'] = LR_fit.predict(X_test)

In [ ]:
#Training and test MAE - Mean Absolute error
print('Training MAE:',mean_absolute_error(dfTR_eval['charges'], dfTR_eval['LR_pred']))
print('Test MAE:',mean_absolute_error(dfTS_eval['charges'], dfTS_eval['LR_pred']))

In [ ]:
#Training and test RMSE - Root Mean Square Error
print('Training RMSE:',math.sqrt(mean_squared_error(dfTR_eval['charges'], dfTR_eval['LR_pred'])))
print('Test RMSE:',math.sqrt(mean_squared_error(dfTS_eval['charges'], dfTS_eval['LR_pred'])))

In [5]:
#Training and test r^2 
print('Training R2:',r2_score(dfTR_eval['charges'], dfTR_eval['LR_pred']))
print('Test R2:',r2_score(dfTS_eval['charges'], dfTS_eval['LR_pred']))

MSE(TR, poly) = 0.004880797137341219
MSE(TS, poly) = 0.005359776599030808
MAE(TR, poly) = 0.041669632834955624
MAE(TS, poly) = 0.043841128675839974


In [6]:
# First, create the basis functions using SplineTransformer
n_knots = 25 # number of "breaking points"
degree = 3 # order of the basis polynomials

splt = SplineTransformer(n_knots=n_knots, degree=degree).fit(X_train)
X_train_splt = splt.transform(X_train)

# fit the B-spline using ridge regression
ridge_splt = Ridge(alpha=0.1)
ridge_splt.fit(X_train_splt, y_train)

# show coefs
# plt.figure(figsize=(15, 3))
# plt.bar(range(n_knots+2), ridge_splt.coef_)
# plt.title(f'Intercept: {ridge_splt.intercept_}')
# plt.xlabel('Base id')
# plt.ylabel('Coef.')
# plt.grid()
# plt.show()

Ridge(alpha=0.1)

In [7]:
# estimations using the spline
y_train_spline = ridge_splt.predict(X_train_splt)

X_test_splt = splt.transform(X_test)
y_test_spline = ridge_splt.predict(X_test_splt)

# errors
mse_tr_spline = mean_squared_error(y_train_spline, y_train)
mse_ts_spline = mean_squared_error(y_test_spline, y_test)

mae_tr_spline = mean_absolute_error(y_train_spline, y_train)
mae_ts_spline = mean_absolute_error(y_test_spline, y_test)

print(f'MSE(TR, spline) = {mse_tr_spline}')
print(f'MSE(TS, spline) = {mse_ts_spline}')
print(f'MAE(TR, spline) = {mae_tr_spline}')
print(f'MAE(TS, spline) = {mae_ts_spline}')

MSE(TR, spline) = 0.005087265579134734
MSE(TS, spline) = 0.005306321790805033
MAE(TR, spline) = 0.04771968010542406
MAE(TS, spline) = 0.049284292080676924


### Stacking

In [8]:
degree_poly = 15

n_knots_spline = 25 # number of "breaking points"
degree_basis_spline = 3 # order of the basis polynomials

# grid in min_impurity x min_samples_leaf x min_samples_split
param_tree = {'DT__min_impurity_decrease': np.arange(0, 0.001 * np.var(y_train),0.001), # Minimum impurity to decrease in each split
         'DT__min_samples_leaf': np.arange(1,10,1), # Minimum number of obs in a terminal node
         'DT__min_samples_split':  np.arange(1,10,1)} # Minimum number of obs in node to keep cutting

pipe_tree = Pipeline(steps=[('DT', DecisionTreeRegressor(criterion='squared_error',  # impurity measure: variance reduction
                                                         random_state=150))]) # For replication

nFolds_tree = 10

## set of estimators to be stacked
estimators = [
    ('poly', make_pipeline(
        PolynomialFeatures(degree_poly), 
        Ridge(alpha=1e-3))
    ),
    ('spline', make_pipeline(
        SplineTransformer(n_knots=n_knots, degree=degree),
        Ridge(alpha=1e-3))
    ),
    ('regtree', make_pipeline(
                GridSearchCV(estimator= pipe_tree,
                            param_grid=param_tree, # Defined grid to search in
                            n_jobs=-1, # Number of cores to use (parallelize)
                            cv=nFolds_tree)) # Number of Folds
    )
]

estimators

[('poly',
  Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=15)),
                  ('ridge', Ridge(alpha=0.001))])),
 ('spline',
  Pipeline(steps=[('splinetransformer', SplineTransformer(n_knots=25)),
                  ('ridge', Ridge(alpha=0.001))])),
 ('regtree',
  Pipeline(steps=[('gridsearchcv',
                   GridSearchCV(cv=10,
                                estimator=Pipeline(steps=[('DT',
                                                           DecisionTreeRegressor(random_state=150))]),
                                n_jobs=-1,
                                param_grid={'DT__min_impurity_decrease': array([0.]),
                                            'DT__min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                            'DT__min_samples_split': array([1, 2, 3, 4, 5, 6, 7, 8, 9])}))]))]

In [9]:
weights = [0.1, 0.2, 0.7]
stacked_reg = VotingRegressor(estimators = estimators, weights = weights)
stacked_reg = stacked_reg.fit(X_train, y_train)
stacked_reg.get_params()

c:\Users\mdavila\anaconda3\envs\ML\Lib\site-packages\sklearn\linear_model\_ridge.py:250: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\mdavila\anaconda3\envs\ML\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
90 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\mdavila\anaconda3\envs\ML\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\mdavila\anaconda3\envs\ML\Lib\site-packages\sklearn\base.py", line 1151, i

{'estimators': [('poly',
   Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=15)),
                   ('ridge', Ridge(alpha=0.001))])),
  ('spline',
   Pipeline(steps=[('splinetransformer', SplineTransformer(n_knots=25)),
                   ('ridge', Ridge(alpha=0.001))])),
  ('regtree',
   Pipeline(steps=[('gridsearchcv',
                    GridSearchCV(cv=10,
                                 estimator=Pipeline(steps=[('DT',
                                                            DecisionTreeRegressor(random_state=150))]),
                                 n_jobs=-1,
                                 param_grid={'DT__min_impurity_decrease': array([0.]),
                                             'DT__min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                             'DT__min_samples_split': array([1, 2, 3, 4, 5, 6, 7, 8, 9])}))]))],
 'n_jobs': None,
 'verbose': False,
 'weights': [0.1, 0.2, 0.7],
 'poly': Pipeline(steps=[('polynomi

In [10]:
# estimations using the stacked models
y_train_stack = stacked_reg.predict(X_train)
y_test_stack = stacked_reg.predict(X_test)

# errors
mse_tr_stack = mean_squared_error(y_train_stack, y_train)
mse_ts_stack = mean_squared_error(y_test_stack, y_test)

mae_tr_stack = mean_absolute_error(y_train_stack, y_train)
mae_ts_stack = mean_absolute_error(y_test_stack, y_test)

print(f'MSE(TR, stack) = {mse_tr_stack}')
print(f'MSE(TS, stack) = {mse_ts_stack}')
print(f'MAE(TR, stack) = {mae_tr_stack}')
print(f'MAE(TS, stack) = {mae_ts_stack}')

MSE(TR, stack) = 0.0032177129378747984
MSE(TS, stack) = 0.004713365320588808
MAE(TR, stack) = 0.03607491137779835
MAE(TS, stack) = 0.04236636090248357
